# Teste Geracao Filtro

- Notebook para geracao dos coeficientes do filtro utilizado no projeto.
- Filtro utilizado:  
    - Filtro digital **IIR** _(Infinite Impulse Response)_ do tipo **Biquad Cascade**, utilizando as funções otimizadas da biblioteca CMSIS-DSP da ARM (no Microcontrolador).
    - Tipo **Chebyshev**
    - Frequencia de amostragem do sinal: **48 kHz**
    - Passa faixa:
        - Frequencia de corte inferior: **20 Hz**
        - Frequencia de corte superior: **20 kHz**


In [ ]:
import scipy.signal as signal
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


In [ ]:
# Especificações do filtro baseadas no artigo
FS_FILTER = 48000      # Frequência de amostragem do filtro em Hz
LOWCUT = 100.0          # Frequência de corte inferior em Hz
HIGHCUT = 20000.0      # Frequência de corte superior em Hz
ORDER = 4              # Ordem do filtro
FILTER_TYPE = 'cheby1' # Opções: 'butter', 'cheby1', 'cheby2'
RP = 0.5               # Ripple máximo na banda de passagem em dB (apenas para Chebyshev)


In [ ]:
def design_filter():
    """Projeta o filtro e retorna os coeficientes em formato SOS."""
    
    # Normaliza as frequências de corte para a frequência de Nyquist (FS/2)
    nyquist = 0.5 * FS_FILTER
    low = LOWCUT / nyquist
    high = HIGHCUT / nyquist

    # Projeta o filtro e obtém os coeficientes em formato SOS (Second-Order Sections)
    # 'sos' é uma matriz de formato [n_sections, 6]
    # Cada linha é [b0, b1, b2, a0, a1, a2]
    
    if FILTER_TYPE == 'butter':
        sos = signal.butter(ORDER, [low, high], btype='band', output='sos')
    elif FILTER_TYPE == 'cheby1':
        # Chebyshev Tipo 1: Roll-off mais acentuado, com ripple na banda de passagem.
        sos = signal.cheby1(ORDER, RP, [low, high], btype='band', output='sos')
    elif FILTER_TYPE == 'cheby2':
        sos = signal.cheby2(ORDER, RP, [low, high], btype='band', output='sos')
    else:
        raise ValueError("Tipo de filtro não suportado. Use 'butter', 'cheby1' ou 'cheby2'.")

    # A biblioteca CMSIS-DSP usa coeficientes para a forma de transferência direta I (Direct Form I).
    # A estrutura de dados dela é [b0, b1, b2, a1, a2] para cada estágio. Note que a0 é sempre 1 e omitido,
    # e os sinais de a1 e a2 são invertidos.
    num_stages = sos.shape[0]
    coeffs_cmsis = np.zeros(num_stages * 5)

    for i in range(num_stages):
        b0, b1, b2, a0, a1, a2 = sos[i]
        coeffs_cmsis[i*5 + 0] = b0
        coeffs_cmsis[i*5 + 1] = b1
        coeffs_cmsis[i*5 + 2] = b2
        coeffs_cmsis[i*5 + 3] = -a1 # Inverte o sinal de a1
        coeffs_cmsis[i*5 + 4] = -a2 # Inverte o sinal de a2

    # Exibe os coeficientes no formato para copiar e colar em C
    print(f"// Coeficientes do Filtro Passa-Faixa {FILTER_TYPE.capitalize()} de Ordem {ORDER}")
    print(f"// Fs={FS_FILTER}Hz, Fc=[{LOWCUT}Hz, {HIGHCUT}Hz]")
    print(f"#define NUM_STAGES {num_stages}")
    print("float32_t pCoeffs[NUM_STAGES * 5] = {")
    for i, c in enumerate(coeffs_cmsis):
        print(f"    {c:.8f}f,", end='')
        if (i + 1) % 5 == 0:
            print("")
    print("};")

    return sos

In [ ]:
def visualize_filter_effect(wav_filepath):
    """
    Carrega um arquivo .wav, aplica o filtro e plota os resultados
    nos domínios do tempo e da frequência para análise.
    """
    try:
        # --- 2. Carregar o Arquivo de Áudio ---
        fs_wav, audio_data = wavfile.read(wav_filepath)
        print(f"Arquivo '{wav_filepath}' carregado com sucesso.")
        print(f"Frequência de amostragem do áudio: {fs_wav} Hz")

        if fs_wav != FS_FILTER:
            print(f"ERRO: A frequência de amostragem do áudio ({fs_wav} Hz) é diferente da do filtro ({FS_FILTER} Hz).")
            print("Por favor, use um arquivo de áudio com a amostragem correta.")
            return

        if np.issubdtype(audio_data.dtype, np.integer):
             audio_data = audio_data / np.iinfo(audio_data.dtype).max
        
        if audio_data.ndim > 1:
            print("Áudio estéreo detectado. Usando apenas o canal esquerdo.")
            audio_data = audio_data[:, 0]
            
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado em '{wav_filepath}'")
        return
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo: {e}")
        return

    # --- 3. Aplicar o Filtro ao Sinal ---
    print(f"Projetando e aplicando o filtro {FILTER_TYPE.capitalize()}...")
    sos_coeffs = design_filter()
    audio_filtered = signal.sosfilt(sos_coeffs, audio_data)
    print("Filtragem concluída.")

    # --- 4. Preparar dados para Visualização ---
    time_axis = np.arange(len(audio_data)) / fs_wav
    n_fft = len(audio_data)
    fft_original = np.fft.fft(audio_data, n_fft)
    fft_filtered = np.fft.fft(audio_filtered, n_fft)
    freq_axis = np.fft.fftfreq(n_fft, d=1/fs_wav)
    half_n_fft = n_fft // 2
    positive_freq_axis = freq_axis[:half_n_fft]
    db_fft_original = 20 * np.log10(np.abs(fft_original[:half_n_fft]))
    db_fft_filtered = 20 * np.log10(np.abs(fft_filtered[:half_n_fft]))

    # --- 5. Gerar os Gráficos Comparativos ---
    plt.style.use('seaborn-v0_8-whitegrid')
    
    fig = plt.figure(figsize=(16, 12))
    gs = gridspec.GridSpec(3, 2, figure=fig)
    fig.suptitle(f'Análise do Filtro Passa-Faixa ({LOWCUT}-{HIGHCUT} Hz) no arquivo: {wav_filepath.split("/")[-1]}', fontsize=16)

    ax_orig_time = fig.add_subplot(gs[0, 0])
    ax_filt_time = fig.add_subplot(gs[0, 1])
    ax_orig_freq = fig.add_subplot(gs[1, 0])
    ax_filt_freq = fig.add_subplot(gs[1, 1])
    ax_filt_resp_full = fig.add_subplot(gs[2, 0])
    ax_filt_resp_zoom = fig.add_subplot(gs[2, 1])

    # Gráficos de Tempo e Frequência do Sinal
    ax_orig_time.plot(time_axis, audio_data, label='Sinal Original', color='blue', alpha=0.8)
    ax_orig_time.set_title('Sinal Original (Domínio do Tempo)')
    ax_orig_time.set_xlabel('Tempo (s)'); ax_orig_time.set_ylabel('Amplitude'); ax_orig_time.legend()

    ax_filt_time.plot(time_axis, audio_filtered, label='Sinal Filtrado', color='green', alpha=0.8)
    ax_filt_time.set_title('Sinal Filtrado (Domínio do Tempo)')
    ax_filt_time.set_xlabel('Tempo (s)'); ax_filt_time.legend()

    ax_orig_freq.plot(positive_freq_axis, db_fft_original, label='Espectro Original', color='blue')
    ax_orig_freq.set_title('Espectro do Sinal Original (FFT)')
    ax_orig_freq.set_xlabel('Frequência (Hz)'); ax_orig_freq.set_ylabel('Magnitude (dB)'); ax_orig_freq.legend()
    y_lim_freq = (np.min(db_fft_original) - 10, np.max(db_fft_original) + 5)
    ax_orig_freq.set_ylim(y_lim_freq)

    ax_filt_freq.plot(positive_freq_axis, db_fft_filtered, label='Espectro Filtrado', color='green')
    ax_filt_freq.set_title('Espectro do Sinal Filtrado (FFT)')
    ax_filt_freq.set_xlabel('Frequência (Hz)'); ax_filt_freq.legend()
    ax_filt_freq.set_ylim(y_lim_freq)
    
    # Gráfico 6 e 7: Resposta de Frequência do Filtro (Completa e Zoom)
    w, h = signal.sosfreqz(sos_coeffs, worN=16384, fs=FS_FILTER)
    db_h = 20 * np.log10(np.abs(h))
    
    # Gráfico Completo
    ax_filt_resp_full.plot(w, db_h, label=f'Filtro {FILTER_TYPE.capitalize()}', color='purple')
    ax_filt_resp_full.set_title('Resposta de Frequência do Filtro (Completa)')
    ax_filt_resp_full.set_xlabel('Frequência (Hz)'); ax_filt_resp_full.set_ylabel('Ganho (dB)')
    ax_filt_resp_full.axvline(LOWCUT, color='red', linestyle='--', alpha=0.7, label=f'Corte Inferior ({LOWCUT} Hz)')
    ax_filt_resp_full.axvline(HIGHCUT, color='red', linestyle='--', alpha=0.7)
    ax_filt_resp_full.set_ylim(-80, 5)
    ax_filt_resp_full.set_xlim(0, FS_FILTER / 2)
    ax_filt_resp_full.legend()

    # Gráfico com Zoom na Frequência Baixa
    ax_filt_resp_zoom.plot(w, db_h, label=f'Filtro {FILTER_TYPE.capitalize()}', color='purple')
    ax_filt_resp_zoom.set_title('Resposta de Frequência (Zoom < 500 Hz)')
    ax_filt_resp_zoom.set_xlabel('Frequência (Hz)');
    ax_filt_resp_zoom.axvline(LOWCUT, color='red', linestyle='--', alpha=0.7, label=f'Corte Inferior ({LOWCUT} Hz)')
    ax_filt_resp_zoom.set_xlim(0, 500)
    ax_filt_resp_zoom.set_ylim(-80, 5)
    ax_filt_resp_zoom.grid(True, which='both')
    ax_filt_resp_zoom.legend()
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


In [ ]:
if __name__ == '__main__':
    # Peça ao usuário para fornecer o caminho do arquivo .wav
    # Use áudios gravados pelo seu setup para ter uma representação fiel.
    # Exemplo de caminho: "C:/Users/SeuNome/Documentos/TCC/audios/furadeira_saudavel.wav"
    # Ou simplesmente: "furadeira_saudavel.wav" se estiver na mesma pasta do script.
    
    print("Este script carrega um arquivo .wav, aplica um filtro e exibe os resultados.")
    print(f"Tipo de filtro configurado: {FILTER_TYPE.capitalize()}")
    file_path = input("\nPor favor, insira o caminho para o arquivo .wav (com amostragem de 48000 Hz): ")
    visualize_filter_effect(file_path)